In [22]:
from pyopensky.rest import REST

In [23]:
rest = REST()
rest.states()

2025-11-26 16:13:17,383 - INFO - HTTP Request: GET https://opensky-network.org/api/states/all "HTTP/1.1 200 "


,icao24,callsign,origin_country,last_position,timestamp,longitude,latitude,altitude,onground,groundspeed,track,vertical_rate,sensors,geoaltitude,squawk,spi,position_source
0,e49406,GLO9919,Brazil,2025-11-26 21:12:52+00:00,2025-11-26 21:12:52+00:00,-56.7281,-20.2225,9753.6,False,221.65,292.52,0.0,None,10347.96,<NA>,False,0
1,4b1815,SWR1KV,Switzerland,2025-11-26 21:12:53+00:00,2025-11-26 21:12:53+00:00,10.0312,49.8776,11879.58,False,208.0,23.78,0.0,None,11727.18,3033,False,0
2,a6aeb6,N53GM,United States,2025-11-26 21:12:50+00:00,2025-11-26 21:12:52+00:00,-95.7359,40.2523,1722.12,False,78.24,346.69,0.65,None,1729.74,<NA>,False,0
3,ab2970,N8180D,United States,2025-11-26 21:12:52+00:00,2025-11-26 21:12:53+00:00,-83.0711,32.7698,3322.32,False,85.32,159.16,0.0,None,3390.9,5705,False,0
4,4b1819,SWR24C,Switzerland,2025-11-26 21:12:52+00:00,2025-11-26 21:12:53+00:00,2.8029,49.6087,10363.2,False,205.55,314.09,0.33,None,10393.68,3045,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,0ac9c3,AVA8479,Colombia,2025-11-26 21:08:22+00:00,2025-11-26 21:08:23+00:00,-81.2832,12.4402,3528.06,False,208.21,110.54,7.15,None,3718.56,<NA>,False,0
10076,a9607d,,United States,NaT,2025-11-26 21:12:25+00:00,<NA>,<NA>,<NA>,False,223.28,348.3,0.0,None,<NA>,<NA>,False,0
10077,ac90e8,N909DB,United States,2025-11-26 21:12:53+00:00,2025-11-26 21:12:53+00:00,-104.5415,38.9237,2286.0,False,48.34,343.3,1.3,None,2331.72,<NA>,False,0
10078,a41b89,DAL1169,United States,2025-11-26 21:12:53+00:00,2025-11-26 21:12:53+00:00,-76.3356,38.7911,5707.38,False,238.46,75.12,-8.45,None,5745.48,7557,False,0


In [24]:
from pyopensky.rest import REST
import json
import time
import kafka
from kafka import KafkaProducer
import logging
import sys
import plotly.express as px
import pandas as pd
from sklearn.ensemble import IsolationForest

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout)
logger = logging.getLogger(__name__)

rest = REST()

def get_current_states():
    states = rest.states()
    return states

def get_coordinates(states):
    return states[["callsign","latitude", "longitude"]].to_dict("records")

def iso_tree():
    # Get states and prepare data
    states = get_current_states()
    df = states[["callsign", "latitude", "longitude", "altitude", "groundspeed", "vertical_rate"]].dropna()

    # Train Isolation Forest
    model = IsolationForest(contamination=0.01, random_state=42)
    df["anomaly"] = model.fit_predict(df[["latitude", "longitude", "altitude", "groundspeed", "vertical_rate"]])

    # Filter anomalies (-1 = anomaly, 1 = normal)
    anomalies = df[df["anomaly"] == -1]
    print(f"Found {len(anomalies)} anomalies out of {len(df)} flights")

    # Visualize with anomalies highlighted
    df["is_anomaly"] = df["anomaly"].map({1: "Normal", -1: "Anomaly"})
    fig = px.scatter_geo(df, lat="latitude", lon="longitude", 
                        color="is_anomaly", 
                        color_discrete_map={"Normal": "blue", "Anomaly": "red"},
                        projection="natural earth",
                        title="Flight Anomaly Detection")
    fig.show()

def main():
        states = get_current_states()
        logger.info(f"Current states: {states}")
        coords = get_coordinates(states)
        logger.info(f"Coordinates: {coords}")
        

        df = pd.DataFrame(coords)
        fig = px.scatter_geo(df, lat="latitude", lon="longitude", projection="natural earth")
        fig.show()
        
        iso_tree()

if __name__ == "__main__":
    main()

2025-11-26 16:13:18,512 - INFO - HTTP Request: GET https://opensky-network.org/api/states/all "HTTP/1.1 200 "
2025-11-26 16:13:19,236 - INFO - Current states:        icao24 callsign origin_country             last_position  \
0      e49406  GLO9919         Brazil 2025-11-26 21:12:57+00:00   
1      4b1815   SWR1KV    Switzerland 2025-11-26 21:13:09+00:00   
2      a6aeb6    N53GM  United States 2025-11-26 21:13:05+00:00   
3      ab2970   N8180D  United States 2025-11-26 21:13:09+00:00   
4      4b1819   SWR24C    Switzerland 2025-11-26 21:13:09+00:00   
...       ...      ...            ...                       ...   
10084  0ac9c3  AVA8479       Colombia 2025-11-26 21:08:22+00:00   
10085  a9607d           United States                       NaT   
10086  ac90e8   N909DB  United States 2025-11-26 21:13:09+00:00   
10087  a41b89  DAL1169  United States 2025-11-26 21:13:09+00:00   
10088  c067ae   ACA522         Canada 2025-11-26 21:13:09+00:00   

                      timestamp  lon

2025-11-26 16:13:19,447 - INFO - HTTP Request: GET https://opensky-network.org/api/states/all "HTTP/1.1 200 "
Found 91 anomalies out of 9057 flights


In [25]:
print(anomalies[["callsign", "latitude", "longitude", "altitude", "groundspeed"]])

     callsign  latitude  longitude  altitude  groundspeed
170   AZU8706   25.5443   -78.4997   7840.98       218.59
545     TEX09  -39.8043    175.302   4312.92        93.17
1540  SMASH24    34.026   -97.8372   3329.94       115.28
2474   N391TP   31.4242   -85.1081   2461.26        45.34
3271   LFA315   28.8377   -81.7908  30540.96        35.21
5307   ETH715   56.1217    18.7804  11879.58        37.53
7611   N9454F   21.5863  -158.1737   3726.18        72.72
7714  SWA2382   38.4654   -93.9578    6591.3       182.78
8491  GHOST21   34.6023     -97.89   3451.86        84.12
8650   N211DS   29.3792   -81.2799   1196.34        37.86


In [26]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Get states and prepare data
states = get_current_states()
df = states[["callsign", "latitude", "longitude", "altitude", "groundspeed", "vertical_rate"]].dropna()

# Scale features (important for DBSCAN)
features = df[["latitude", "longitude", "altitude", "groundspeed", "vertical_rate"]]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Train DBSCAN
dbscan = DBSCAN(eps=2, min_samples=5)
df["cluster"] = dbscan.fit_predict(scaled_features)

# Anomalies are labeled as -1 (noise points)
anomalies = df[df["cluster"] == -1]
print(f"Found {len(anomalies)} anomalies out of {len(df)} flights")

# Visualize
df["is_anomaly"] = df["cluster"].apply(lambda x: "Anomaly" if x == -1 else "Normal")
fig = px.scatter_geo(df, lat="latitude", lon="longitude",
                     color="is_anomaly",
                     color_discrete_map={"Normal": "blue", "Anomaly": "red"},
                     projection="natural earth",
                     title="Flight Anomaly Detection (DBSCAN)")
fig.show()

2025-11-26 16:13:20,042 - INFO - HTTP Request: GET https://opensky-network.org/api/states/all "HTTP/1.1 200 "
Found 5 anomalies out of 9059 flights


In [27]:
print(anomalies[["callsign", "latitude", "longitude", "altitude", "groundspeed"]])

     callsign  latitude  longitude  altitude  groundspeed
1914  SNORT15   33.3646   -99.7763   5120.64       198.78
2462   N391TP   31.4133   -85.2111   2766.06        11.33
2561   N755PJ   30.2234   -99.5332  11635.74       166.46
4906  COBRA26   34.4352   -98.9809    2895.6       150.31
5294   ETH715   56.1217    18.7804  11879.58        37.53


In [28]:
# After running both models
df["both_flagged"] = (df["is_anomaly"] == -1) & (df["cluster"] == -1)
consensus_anomalies = df[df["both_flagged"]]
consensus_anomalies

,callsign,latitude,longitude,altitude,groundspeed,vertical_rate,cluster,is_anomaly,both_flagged


In [29]:
from sklearn.metrics import silhouette_score
score = silhouette_score(scaled_features, df["cluster"])
print(f"Silhouette score: {score}")  # Closer to 1 = better clusters

Silhouette score: 0.7067866382183968


In [30]:
states = get_current_states()
df = states[["callsign", "latitude", "longitude", "altitude", "groundspeed", "vertical_rate"]].dropna()

# Isolation Forest
iso_model = IsolationForest(contamination=0.01, random_state=42)
df["anomaly_if"] = iso_model.fit_predict(df[["latitude", "longitude", "altitude", "groundspeed", "vertical_rate"]])

# DBSCAN
scaler = StandardScaler()
scaled = scaler.fit_transform(df[["latitude", "longitude", "altitude", "groundspeed", "vertical_rate"]])
dbscan = DBSCAN(eps=2, min_samples=5)
df["anomaly_dbscan"] = dbscan.fit_predict(scaled)

# Compare
df["both_flagged"] = (df["anomaly_if"] == -1) & (df["anomaly_dbscan"] == -1)
consensus = df[df["both_flagged"]]
print(f"Consensus anomalies: {len(consensus)}")
print(consensus[["callsign", "latitude", "longitude", "altitude", "groundspeed"]])

2025-11-26 16:32:07,989 - INFO - HTTP Request: GET https://opensky-network.org/api/states/all "HTTP/1.1 200 "
Consensus anomalies: 0
Empty DataFrame
Columns: [callsign, latitude, longitude, altitude, groundspeed]
Index: []
